In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sklearn
from tqdm import tqdm
import random

import data_interface
import mnar_blackout_lds

random.seed(42)
np.random.seed(42)

In [3]:
# Load the data
x_t, m_t, meta = data_interface.load_panel()

In [4]:
evaluation_windows = data_interface.get_eval_windows("data")

In [5]:
from collections import defaultdict

def stratified_month_sampling(data, n_per_month, ts_key="blackout_start"):
    buckets = defaultdict(list)

    for item in data:
        ts = item[ts_key]
        month_key = (ts.year, ts.month)
        buckets[month_key].append(item)

    result = []
    for month_key, items in buckets.items():
        if len(items) < n_per_month:
            picks = random.choices(items, k=n_per_month)
        else:
            picks = random.sample(items, n_per_month)
        result.extend(picks)

    return result


In [6]:
impute_evaluation_windows = [window for window in evaluation_windows if window["test_type"] == "impute"]
forecast_1_evaluation_windows = [window for window in evaluation_windows if window["test_type"] == "forecast" and window["horizon_steps"] == 1]
forecast_3_evaluation_windows = [window for window in evaluation_windows if window["test_type"] == "forecast" and window["horizon_steps"] == 3]
forecast_6_evaluation_windows = [window for window in evaluation_windows if window["test_type"] == "forecast" and window["horizon_steps"] == 6]

In [7]:
impute_evaluation_windows_val = stratified_month_sampling(impute_evaluation_windows, n_per_month=25, ts_key="blackout_start")
forecast_1_evaluation_windows_val = stratified_month_sampling(forecast_1_evaluation_windows, n_per_month=12, ts_key="blackout_start")
forecast_3_evaluation_windows_val = stratified_month_sampling(forecast_3_evaluation_windows, n_per_month=12, ts_key="blackout_start")
forecast_6_evaluation_windows_val = stratified_month_sampling(forecast_6_evaluation_windows, n_per_month=12, ts_key="blackout_start")

In [8]:
evaluation_windows_val = (forecast_1_evaluation_windows_val +
                              forecast_3_evaluation_windows_val +
                              forecast_6_evaluation_windows_val + impute_evaluation_windows_val)

In [9]:
def mask_evaluation_windows(x_t, m_t, evaluation_windows_val, meta):
    x_t_masked = x_t.copy()
    m_t_masked = m_t.copy()

    for window in evaluation_windows_val:
        start_idx = np.where(meta["timestamps"]==window["blackout_start"])[0][0]
        end_idx = np.where(meta["timestamps"]==window["blackout_end"])[0][0]
        detector_idx = np.where(meta["detectors"]==window["detector_id"])[0][0]
        
        x_t_masked[start_idx:end_idx+1, detector_idx] = np.nan
        m_t_masked[start_idx:end_idx+1, detector_idx] = 1
    
    return x_t_masked, m_t_masked

In [10]:
# Prepare training data by masking evaluation windows
x_t_train, m_t_train = mask_evaluation_windows(x_t, m_t, evaluation_windows_val, meta)

In [11]:
model_params = mnar_blackout_lds.MNARParams.init_random(K=9, D=147, seed=42)
model = mnar_blackout_lds.MNARBlackoutLDS(model_params)

In [12]:
em_train_history = model.em_train(x_t, m_t, num_iters=15)


=== EM iteration 1/15 ===
  A norm: 2.965
  Q trace: 4.534
  mean diag(R): 58.312

=== EM iteration 2/15 ===
  A norm: 2.855
  Q trace: 17.436
  mean diag(R): 30.775

=== EM iteration 3/15 ===
  A norm: 2.869
  Q trace: 23.895
  mean diag(R): 28.049

=== EM iteration 4/15 ===
  A norm: 2.875
  Q trace: 27.774
  mean diag(R): 27.733

=== EM iteration 5/15 ===
  A norm: 2.881
  Q trace: 30.633
  mean diag(R): 27.715

=== EM iteration 6/15 ===
  A norm: 2.884
  Q trace: 33.074
  mean diag(R): 27.759

=== EM iteration 7/15 ===
  A norm: 2.887
  Q trace: 35.225
  mean diag(R): 27.794

=== EM iteration 8/15 ===
  A norm: 2.888
  Q trace: 36.840
  mean diag(R): 27.802

=== EM iteration 9/15 ===
  A norm: 2.890
  Q trace: 37.830
  mean diag(R): 27.796

=== EM iteration 10/15 ===
  A norm: 2.891
  Q trace: 38.350
  mean diag(R): 27.783

=== EM iteration 11/15 ===
  A norm: 2.892
  Q trace: 38.475
  mean diag(R): 27.759

=== EM iteration 12/15 ===
  A norm: 2.894
  Q trace: 38.222
  mean diag(R

### Reconstruction and Prediction

In [13]:
ekf_out = model.ekf_forward(x_t_train, m_t_train)
smoother_out = model.rts_smoother(ekf_out)

mu_filt = ekf_out["mu_filt"]
Sigma_filt = ekf_out["Sigma_filt"]
mu_smooth = smoother_out["mu_smooth"]
Sigma_smooth = smoother_out["Sigma_smooth"]

In [14]:
# Evaluate imputation performance

impute_mae_list = []
impute_mse_list = []

for window in tqdm(impute_evaluation_windows_val):
    if window["test_type"] == "impute":
        start_idx = np.where(meta["timestamps"]==window["blackout_start"])[0][0]
        end_idx = np.where(meta["timestamps"]==window["blackout_end"])[0][0]

        detector_idx = np.where(meta["detectors"]==window["detector_id"])[0][0]

        eval_x_t = x_t[start_idx:end_idx+1].copy()
        eval_mu_smooth = mu_smooth[start_idx:end_idx+1]
        eval_Sigma_smooth = Sigma_smooth[start_idx:end_idx+1]

        reconstruct_x_t, _ = model.reconstruct_from_smoother(eval_mu_smooth, eval_Sigma_smooth)

        mae = sklearn.metrics.mean_absolute_error(reconstruct_x_t[:, detector_idx], eval_x_t[:, detector_idx])
        mse = sklearn.metrics.mean_squared_error(reconstruct_x_t[:, detector_idx], eval_x_t[:, detector_idx])
        rmse = np.sqrt(mse)
        
        impute_mae_list.append([mae, window["len_steps"]])
        impute_mse_list.append([mse, window["len_steps"]])

final_mae = np.average([item[0] for item in impute_mae_list], weights=[item[1] for item in impute_mae_list])
final_mse = np.average([item[0] for item in impute_mse_list], weights=[item[1] for item in impute_mse_list])
final_rmse = np.sqrt(final_mse)

print("Final MAE (impute):", final_mae)
print("Final MSE (impute):", final_mse)
print("Final RMSE (impute):", final_rmse)

100%|██████████| 300/300 [00:00<00:00, 415.99it/s]

Final MAE (impute): 3.2031141365160125
Final MSE (impute): 24.998220712043523
Final RMSE (impute): 4.999822068038374


In [15]:
# Evaluate forecasting performance

y_actual_1_step, y_forecast_1_step = [], []
y_actual_3_step, y_forecast_3_step = [], []
y_actual_6_step, y_forecast_6_step = [], []

forecast_evaluation_windows_val = (forecast_1_evaluation_windows_val +
                                   forecast_3_evaluation_windows_val +
                                   forecast_6_evaluation_windows_val)

for window in tqdm(forecast_evaluation_windows_val):
    if window["test_type"] == "forecast":
        start_idx = np.where(meta["timestamps"]==window["blackout_start"])[0][0]
        end_idx = np.where(meta["timestamps"]==window["blackout_end"])[0][0]
        blackout_idx = end_idx

        detector_idx = np.where(meta["detectors"]==window["detector_id"])[0][0]

        eval_x_t = x_t[end_idx+1:end_idx+1+int(window["horizon_steps"])].copy()

        forecast_x_t, _ = model.k_step_forecast(mu_filt, Sigma_filt, blackout_idx, k=int(window["horizon_steps"]))
        
        if int(window["horizon_steps"]) == 1:
            y_forecast_1_step.append(forecast_x_t[detector_idx])
            y_actual_1_step.append(eval_x_t[int(window["horizon_steps"])-1, detector_idx])
        elif int(window["horizon_steps"]) == 3:
            y_forecast_3_step.append(forecast_x_t[detector_idx])
            y_actual_3_step.append(eval_x_t[int(window["horizon_steps"])-1, detector_idx])
        elif int(window["horizon_steps"]) == 6:
            y_forecast_6_step.append(forecast_x_t[detector_idx])
            y_actual_6_step.append(eval_x_t[int(window["horizon_steps"])-1, detector_idx])

mae_1_step = sklearn.metrics.mean_absolute_error(y_forecast_1_step, y_actual_1_step)
mse_1_step = sklearn.metrics.mean_squared_error(y_forecast_1_step, y_actual_1_step)
rmse_1_step = np.sqrt(mse_1_step)

print("Evaluation results for forecasting:")
print("\n-----------------------------------")
print("1-step MAE (forecast):", mae_1_step)
print("1-step MSE (forecast):", mse_1_step)
print("1-step RMSE (forecast):", rmse_1_step)


mae_3_step = sklearn.metrics.mean_absolute_error(y_forecast_3_step, y_actual_3_step)
mse_3_step = sklearn.metrics.mean_squared_error(y_forecast_3_step, y_actual_3_step)
rmse_3_step = np.sqrt(mse_3_step)
print("\n-----------------------------------")
print("3-step MAE (forecast):", mae_3_step)
print("3-step MSE (forecast):", mse_3_step)
print("3-step RMSE (forecast):", rmse_3_step)

mae_6_step = sklearn.metrics.mean_absolute_error(y_forecast_6_step, y_actual_6_step)
mse_6_step = sklearn.metrics.mean_squared_error(y_forecast_6_step, y_actual_6_step)
rmse_6_step = np.sqrt(mse_6_step)
print("\n-----------------------------------")
print("6-step MAE (forecast):", mae_6_step)
print("6-step MSE (forecast):", mse_6_step)
print("6-step RMSE (forecast):", rmse_6_step)

100%|██████████| 432/432 [00:00<00:00, 2914.96it/s]

Evaluation results for forecasting:

-----------------------------------
1-step MAE (forecast): 3.316977799779864
1-step MSE (forecast): 30.133898373393908
1-step RMSE (forecast): 5.489435159776815

-----------------------------------
3-step MAE (forecast): 3.968016825578854
3-step MSE (forecast): 41.57623800097983
3-step RMSE (forecast): 6.447963864738995

-----------------------------------
6-step MAE (forecast): 3.437076891691501
6-step MSE (forecast): 31.99566789266958
6-step RMSE (forecast): 5.6564713287233745


In [24]:
x_t[end_idx+1:end_idx+1+int(window["horizon_steps"]), detector_idx].copy()

array([64.56474915, 65.92871343, 65.83385667, 65.22479356, 65.35013493,
       64.80879189])

In [26]:
eval_x_t[int(window["horizon_steps"])-1]

np.float64(64.8087918921252)

In [22]:
forecast_x_t[detector_idx]

np.float64(63.190747908511966)

In [17]:
len(impute_mae_list), len(y_actual_1_step), len(y_actual_3_step), len(y_actual_6_step)

(300, 144, 144, 144)